# Hyperparameter tunning
Vamos a ver como funciona el tunning de hiperparametros.

Empezaremos con el concepto de Cross-Validation

Cargamos ahora las librerías y los datos

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

Hacemos ahora el split de datos para test y train:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

Entrenamos ahora el SVM con los parametros "típicos" y veamos el resultado:

In [ ]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Cross Validation
Cargamos las librerias que nos falta y los datos que hemos usado en anterioridad.

In [ ]:
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=5)

print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Podemos usar otro tipo de evaluaciones:

In [ ]:
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
print(scores)

## Parametric Search

Vamos a usar un SVM para ver como funciona esta opcion.

Empecemos cargando las librerias y el dataset de digitos:

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# Digits dataset
digits = datasets.load_digits()

Dejamos preparado el dataset (hay que pasar las imagenes a 1D)

In [ ]:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

Hacemos ahora una lista con los parametros que queremos investigar, o tunear:

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

Mostramos el resultado

In [ ]:
scores = ['precision', 'recall']

for score in scores:
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("El mejor parametros en el dev test:")
    print()
    print(clf.best_params_)
    print()
    print("Valores de Busqueda:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Information del mejor modelo detallado:")
    print()
    print("El modelo se ha entrenado en todo el dev set")
    print("La puntacion es sobre todo el dev set")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


## Pipelines
Vamos a ver una ultima cosa, los pipelines. Usaremos otro dataset, esta vez de predicci'on de la calidad de un vino.

In [5]:
import pandas as pd 
from sklearn.model_selection import train_test_split


winedf = pd.read_csv('winequality-red.csv',sep=';')

X=winedf.drop(['quality'],axis=1)
Y=winedf['quality']

Investiguemos un poco el dataset:

In [6]:
print(winedf['quality'].value_counts())
winedf.head()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Vamos ahora a crear un pipeline, delante del SVM para hacer la normalizacion:

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

steps = [('scaler', StandardScaler()), ('SVM', SVC())]
pipeline = Pipeline(steps) # define the pipeline object.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=30, stratify=Y)

Vamos a ver ahora como encontrar el valor optimo de la C y la gamma de la SVM:

In [14]:
parameteres = {'SVM__C':[1,10,100], 'SVM__gamma':[1,0.5,0.1,0.01]}
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)
grid.fit(X_train, y_train)
print("score = %3.2f" %(grid.score(X_test,y_test)))
print(grid.best_params_)

score = 0.65
{'SVM__C': 10, 'SVM__gamma': 0.5}
